In [7]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [8]:
class Backpropagation:

    def __init__(self,
                 _no_input_neuron,
                 _hidden_config,
                 _no_output_neuron,
                 _input_data_set,
                 _desired_output,
                 _epoch):
        self.epochs = _epoch

        self.no_of_input_neurons = _no_input_neuron
        self._hidden_config = _hidden_config
        self.no_of_layers = len(_hidden_config)
        self.no_of_hidden_neurons = _hidden_config[0]
        self.no_of_output_neurons = _no_output_neuron

        self.input_data_set = _input_data_set
        self.desired_output = _desired_output
        
        self.eta = 0.03
        
    def generate_weight(self,is_random,_size):
        return np.random.uniform(size=_size) if is_random else np.zeros(_size)
        
    def sigmoid (self,x): 
        return 1/(1 + np.exp(-x)) 
    
    def derivative_(self,x): 
        return x * (1 - x)
    
    def local_field(self,x,w):
        return np.dot(x,w)
    
    def error(self,y):
        return self.desired_output - y
    
    def delta(self,sigma_tic,summed_error,flag = False):
        return summed_error * sigma_tic        
#         return np.dot(sigma_tic,summed_error) if flag else sigma_tic*summed_error

In [3]:
bp = Backpropagation(2,
                     [2],
                     1,
                     np.array([[0,0],[0,1],[1,0],[1,1]]),
                     np.array([[0],[1],[1],[0]]),
                     5000)
Wh = np.random.uniform(size=(bp.no_of_input_neurons, bp._hidden_config[0]))
Wz = np.random.uniform(size=(bp._hidden_config[0],bp.no_of_output_neurons))

for i in range(bp.epochs):
    H = bp.sigmoid(np.dot(bp.input_data_set, Wh))                  # hidden layer results
    Z = bp.sigmoid(np.dot(H, Wz))                  # output layer results
    E = bp.desired_output - Z
#     print E
    dZ = E * bp.derivative_(Z) 
    dH = dZ.dot(Wz.T) * bp.derivative_(H)             # delta H
    Wz +=  0.1 * H.T.dot(dZ)                          # update output layer weights
    Wh +=  0.1 * bp.input_data_set.T.dot(dH) 



# Exercise 3
Investigate the use of back-propagation learning using a sigmoidal nonlinearity to achieve one-toone mappings, as described here:

In [167]:
# Generate data based on minimum and maximum value, n is no. of samples
def get_data(minimum,maximum,n):
    data = np.zeros((1,n))
    for i in range(n):
        data[0,i] = random.uniform(minimum, maximum)
    return data
    
#Training phase
#Wh and Wz are hidden and output layer weights
def training(training_set,desired,Wh,Wz):
    
    epoch_e = []
    squared_error = []
    epoch_count = 0
    while(True):
        #Forward pass
        vh = bp.local_field(training_set.T,Wh)
        sigmoid = bp.sigmoid(vh)
        vo = bp.local_field(sigmoid,Wz)
        y = bp.sigmoid(vo)
        e = desired.T - y
        epoch_e.append(e)
    
        #Backward pass
        dZ = e * bp.derivative_(y) 
        dH = dZ.dot(Wz.T) * bp.derivative_(sigmoid)
        Wz +=  -eta * np.dot(sigmoid.T,dZ)
        Wh +=  -eta * training_set.dot(dH)
        
        if epoch_count>0:
            squared_error.append((epoch_e[epoch_count]**2-epoch_e[epoch_count-1]**2))
            # If average squared error is less than 0.01, we stop adjustment
            if  (np.average(squared_error[epoch_count-1])) < 0.01:
                break

        epoch_count +=1
    print "Training phase"
    print "Number of epochs it took: ", epoch_count
    return (Wh,Wz)

# Test data using weight adjusted during training phase
def testing(test_data,Wh,Wz):
    print "Testing"
    test_vh = bp.local_field(test_data.T,Wh)
    test_sigmoid = bp.sigmoid(test_vh)
    test_vo = bp.local_field(test_sigmoid,Wz)
    output_mapping = bp.sigmoid(test_vo)
    return output_mapping


'''
To compute accuracy: 
    i. We check how much test data is classified correctly, 
     based on weights adjusted during training phase
'''
def compute_accuracy(training_data,test_set,
                     desired,expected,
                     nHidden,nOutput):
    
    
    Wh = np.random.rand(1, nHidden)
    Wz = np.random.rand(nHidden,nOutput)

    Wh,Wz = training(training_data,desired,Wh,Wz)
    print "Adjusted weights from training phase :"
    print "hidden weights ", Wh
    print "Output weights ", Wz
    #we use sample weights for testing
    actual = testing(test_set,Wh,Wz)
    
    
    

In [172]:
#Generate Training data

#get_data takens minimum, maximum, number_of_samples
#f(x) = 1/x
training_set1 = get_data(1,100,200)
desired1 = np.asarray([1.0/x for x in training_set1])

#f(x) = log_10(x)
training_set2 = get_data(1,10,20)
desired2 = np.asarray([np.log10(x) for x in training_set2])

#f(x) = exp(-x)
training_set3 = get_data(1,10,20)
desired3 = np.asarray([np.exp(-x) for x in training_set3])

#f(x) = sin(x)
training_set4 = get_data(1,45,20)
desired4 = np.asarray([np.sin(x) for x in training_set4])

#Generate Test sets
# we generate half numebr of samples for test as compare to training
test_set1 = get_data(1,100,100)
expected1 = np.asarray([1.0/x for x in test_set1])

test_set2 = get_data(1,10,10)
expected2 = np.asarray([np.log10(x) for x in test_set2])

test_set3 = get_data(1,10,10)
expected3 = np.asarray([np.exp(-x) for x in test_set3])

test_set4 = get_data(1,45,10)
expected4 = np.asarray([np.sin(np.radians(x)) for x in test_set4])


nHLayers = 1 #hidden layers
nOutput = 1 #hidden neurons
eta = 0.3 #learning rate
nHidden = [3] # number of hidden  neurons

In [173]:
#i) f(x) = 1/x
for i in range(len(nHidden)):
    compute_accuracy(training_set1,test_set1,desired1,expected1,nHidden[i],nOutput)

Training phase
Number of epochs it took:  2
Adjusted weights from training phase :
hidden weights  [[  0.96595096   0.31428305  12.30265004]]
Output weights  [[ 8.40216971]
 [ 8.14183673]
 [ 6.74096635]]
Testing


In [174]:
#i) f(x) = log(x)
for i in range(len(nHidden)):
    compute_accuracy(training_set2,test_set2,desired2,expected2,nHidden[i],nOutput)

Training phase
Number of epochs it took:  1
Adjusted weights from training phase :
hidden weights  [[ 0.96116125  0.85601473  0.54494088]]
Output weights  [[ 0.98758086]
 [ 1.05909418]
 [ 0.89963497]]
Testing


In [175]:
#i) f(x) = exp(-x)
for i in range(len(nHidden)):
    compute_accuracy(training_set3,test_set3,desired3,expected3,nHidden[i],nOutput)

Training phase
Number of epochs it took:  4
Adjusted weights from training phase :
hidden weights  [[ 0.70079645  0.91027208  0.74596715]]
Output weights  [[ 1.75172832]
 [ 1.15067955]
 [ 1.92467334]]
Testing


In [176]:
#i) f(x) = sin(x)
for i in range(len(nHidden)):
    compute_accuracy(training_set4,test_set4,desired4,expected4,nHidden[i],nOutput)

Training phase
Number of epochs it took:  4
Adjusted weights from training phase :
hidden weights  [[ 0.88332294  0.59444611  0.62668124]]
Output weights  [[ 1.17280332]
 [ 1.77834417]
 [ 1.63295171]]
Testing
